In [160]:
import numpy as np
import pandas as pd
import joblib
import pickle
pd.options.display.max_colwidth = 150

In [3]:
rawdir = 'data/raw'
files = ['claims-2002-2006.xls', 'claims-2007-2009.xls', 'claims-2010-2013.xls',
         'claims-2014.xls', 'claims-2015.xlsx', 'claims-2016.csv', 'claims-2017.csv']
filelist = [f'{rawdir}/{file}' for file in files]

In [4]:
filelist

['data/raw/claims-2002-2006.xls',
 'data/raw/claims-2007-2009.xls',
 'data/raw/claims-2010-2013.xls',
 'data/raw/claims-2014.xls',
 'data/raw/claims-2015.xlsx',
 'data/raw/claims-2016.csv',
 'data/raw/claims-2017.csv']

In [6]:
d = {}
for i, file in enumerate(filelist):
    if 'xls' in file:
        d[i] = pd.read_excel(file)
    if 'csv' in file:
        d[i] = pd.read_csv(file)
        d[i] = d[i].iloc[:, 1:]
    else:
        assert "Error"

In [156]:
df = pd.concat(d, axis=0, sort=False, ignore_index=True)

### Rename columns

col_names = ['claim_number', 'date_received', 'incident_date', 'airport_code', 'airport_name',
             'airline', 'claim_type', 'claim_site', 'item', 'claim_amount', 'status',
             'close_amount', 'disposition', 'item_category']

col_mapper = dict(zip(df.columns, col_names))
df.rename(columns=col_mapper, inplace=True)

### Clean and drop columns

# combine the Item and Item Category column into just Item Category
df['item_category'].fillna(value=df['item'], inplace=True)
df.item_category = df.item_category.astype('str')

# drop item category (only doing item_category now) and airport_name
df.drop(columns=['item', 'airport_name'], inplace=True)

# Set DateTime columns
df['incident_date'] = pd.to_datetime(df['incident_date'], errors='coerce')
df['date_received'] = pd.to_datetime(df['date_received'], errors='coerce')
df.dropna(subset=['date_received', 'incident_date'], inplace=True)

# df['date_received'] = pd.to_datetime(df['date_received'])
# df['incident_date'] = pd.to_datetime(df['incident_date'])

# Limit to 3 outcomes of interest (and clean)
df.disposition = df.disposition.str.replace('D eny|Den y|D en y|De ny',
                           'Deny', regex=True)
df.disposition = df.disposition.str.replace('Approv e in Full',
                           'Approve in Full', regex=True)

df = df[(df['disposition'] == "Approve in Full") |
        (df['disposition'] == "Settle") |
        (df['disposition'] == "Deny")]

# Fix disrepancies in close_amount based on other information. Assuming disposition of Approve or Deny is correct.
df.loc[(df.close_amount.isna()) & (df.disposition == 'Approve in Full'), 'close_amount'] = df.claim_amount
df.loc[(df.close_amount.isna()) & (df.disposition == 'Deny'), 'close_amount'] = 0

# Format close_amount column
df.close_amount = df.close_amount.astype('str')
df.close_amount = df.close_amount.str.replace('$','', regex=False)
df = df[df.close_amount.str.contains('[^a-zA-Z]')]
df.close_amount = df.close_amount.str.replace(',', '',regex=False)
df.close_amount = df.close_amount.str.replace(' ', '',regex=False)
df.close_amount = df.close_amount.str.replace('-', '0',regex=False)
df.close_amount = df.close_amount.astype('float64')

# Drop bad dates
df = df[(df.date_received.dt.year >= 2002) & (df.date_received.dt.year <= 2017)]
df = df[(df.incident_date.dt.year >= 2001) & (df.incident_date.dt.year <= 2017)]

df = df.sort_values(by='date_received', ascending=False)

# strip and clean a bit
df.airline = df.airline.str.strip()
df.airport_code = df.airport_code.str.strip()

In [478]:
cleandir = 'data/clean'

In [10]:
type(df)

pandas.core.frame.DataFrame

In [11]:
# joblib.dump(df, f'{cleandir}/clean_df.joblib', compress=3)

## From this point forward, this is cleaning for 2002-2017 CLASSIFIER and not 2002-2009 Regressor

In [157]:
df.drop(columns=['claim_number', 'status', 'claim_amount'], inplace=True)

# clean 'claim_type'
df.claim_type = df.claim_type.str.replace(' ','')
df.claim_type = df.claim_type.str.replace('PassengerPropertyLoss', 'PropertyLoss')
df = df[
       (df.claim_type == 'PropertyLoss') | (df.claim_type == 'PropertyDamage') | 
       (df.claim_type == 'PersonalInjury') | (df.claim_type == 'EmployeeLoss(MPCECA)') | 
       (df.claim_type == 'PassengerTheft') | (df.claim_type == 'MotorVehicle')
        ]
# clean 'claim_site'
df.claim_site = df.claim_site.str.replace('-', 'Other')
df.claim_site = df.claim_site.str.replace('C heckpoint', 'Checkpoint')
df.claim_site = df.claim_site.str.replace('Checked  Baggage', 'Checked Baggage')

df = df[(df.claim_site != 'PreOtherCheck')]
df = df[(df.claim_site != 'BusStation')]

df.item_category = df.item_category.astype('str')

# simplify categories with subcategories into only normal categories
df.item_category = df.item_category.str.replace('\s-[^;]*', '')
df.item_category = df.item_category.str.replace('\s\([^;]*', '')

# Drop weird misclassified beginners
df = df[df.item_category.str.contains('^[^/]')]
df = df[df.item_category.str.contains('^[^&]')]

# Strip whitespace
df.item_category = df.item_category.str.replace('\s;',';')
df.item_category = df.item_category.str.replace('[^.*]{1}$\s','')
df.item_category = df.item_category.str.replace('^;\s','')

# Fix one off typos
df.item_category = df.item_category.str.replace('Jew$', 'Jewelry & Watches')
df.item_category = df.item_category.str.replace('Home Décor', 'Home Decor')

df.item_category = df.item_category.str.replace('Computer$', 'Computer & Accessories')
df.item_category = df.item_category.str.replace('Computer peripheral$', 'Computer & Accessories')
df.item_category = df.item_category.str.replace('Hunting & Fi$', 'Hunting & Fishing')
df.item_category = df.item_category.str.replace('Jewelry$', 'Jewelry & Watches')
df.item_category = df.item_category.str.replace('Jewelry & Watch$', 'Jewelry & Watches')
df.item_category = df.item_category.str.replace('Othe$', 'Other')
df.item_category = df.item_category.str.replace('Travel A$', 'Travel Accessories')
df.item_category = df.item_category.str.replace('Travel Accessorie$', 'Travel Accessories')
df.item_category = df.item_category.str.replace('Tools & Tool Boxes$', 'Tool Chests & Tool Boxes')
df.item_category = df.item_category.str.replace('Tools$', 'Tools & Home Improvement Supplies')
df.item_category = df.item_category.str.replace('Toys$', 'Toys & Games')
df.item_category = df.item_category.str.replace('Personal Accesso$', 'Personal Accessories')
df.item_category = df.item_category.str.replace('Personal Accessor$', 'Personal Accessories')
df.item_category = df.item_category.str.replace('Personal Accessories$', 'Personal Accessories')
df.item_category = df.item_category.str.replace('Personal E$', 'Personal Electronics')
df.item_category = df.item_category.str.replace('Personal Ele$', 'Personal Electronics')
df.item_category = df.item_category.str.replace('Personal Electronic$', 'Personal Electronics')
df.item_category = df.item_category.str.replace('Medica$', 'Medical/Science')
df.item_category = df.item_category.str.replace('Medical/S$', 'Medical/Science')
df.item_category = df.item_category.str.replace('Medica$', 'Medical/Science')
df.item_category = df.item_category.str.replace('Sporting$', 'Sporting Equipment & Supplies')
df.item_category = df.item_category.str.replace('Cosmetics$', 'Cosmetics & Grooming')
df.item_category = df.item_category.str.replace('Drapery and Curtain Rods, Venetian Blinds$','Drapes')
df.item_category = df.item_category.str.replace('Food items$', 'Food & Drink')
df.item_category = df.item_category.str.replace('H$', '')
df.item_category = df.item_category.str.replace('Hunting & Fishing Items$', 'Hunting & Fishing')
df.item_category = df.item_category.str.replace('Musical Instruments$', 'Musical Instruments & Accessories')
df.item_category = df.item_category.str.replace('Automobile Parts$', 'Automobile Parts & Accessories')
df.item_category = df.item_category.str.replace('Audio$', 'Audio/Video')

df.item_category = df.item_category.str.replace('Computer;', 'Computer & Accessories;')
df.item_category = df.item_category.str.replace('Computer peripherals;', 'Computer & Accessories;')
df.item_category = df.item_category.str.replace('Hunting & Fi;', 'Hunting & Fishing;')
df.item_category = df.item_category.str.replace('Jewelry;', 'Jewelry & Watches;')
df.item_category = df.item_category.str.replace('Jewelry & Watch;', 'Jewelry & Watches;')
df.item_category = df.item_category.str.replace('Othe;', 'Other;')
df.item_category = df.item_category.str.replace('Travel A;', 'Travel Accessories;')
df.item_category = df.item_category.str.replace('Travel Accessorie;', 'Travel Accessories;')
df.item_category = df.item_category.str.replace('Tools & Tool Boxes;', 'Tool Chests & Tool Boxes;')
df.item_category = df.item_category.str.replace('Tools;', 'Tools & Home Improvement Supplies;')
df.item_category = df.item_category.str.replace('Toys;', 'Toys & Games;')
df.item_category = df.item_category.str.replace('Personal Accesso;', 'Personal Accessories;')
df.item_category = df.item_category.str.replace('Personal Accessor;', 'Personal Accessories;')
df.item_category = df.item_category.str.replace('Personal E;', 'Personal Electronics;')
df.item_category = df.item_category.str.replace('Personal Ele;', 'Personal Electronics;')
df.item_category = df.item_category.str.replace('Personal Electronic;', 'Personal Electronics;')
df.item_category = df.item_category.str.replace('Medica;', 'Medical/Science;')
df.item_category = df.item_category.str.replace('Medical/S;', 'Medical/Science;')
df.item_category = df.item_category.str.replace('Medica;', 'Medical/Science;')
df.item_category = df.item_category.str.replace('Sporting;', 'Sporting Equipment & Supplies;')
df.item_category = df.item_category.str.replace('Cosmetics;', 'Cosmetics & Grooming;')
df.item_category = df.item_category.str.replace('Drapery and Curtain Rods, Venetian Blinds;','Drapes;')
df.item_category = df.item_category.str.replace('Food items;', 'Food & Drink;')
df.item_category = df.item_category.str.replace('H;', '')
df.item_category = df.item_category.str.replace('Hunting & Fishing Items;', 'Hunting & Fishing;')
df.item_category = df.item_category.str.replace('Musical Instruments;', 'Musical Instruments & Accessories;')
df.item_category = df.item_category.str.replace('Automobile Parts;', 'Automobile Parts & Accessories;')
df.item_category = df.item_category.str.replace('Audio;', 'Audio/Video;')



# Drop about 2500 rows where there is a missing letter for item_type
df = df[df.claim_type != 'PersonalInjury'][df.item_category.str.contains('^[^a-z]')]


/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [221]:
cleandir = './data/clean'
df.to_pickle(f'{cleandir}/clean_for_classifier_df.pkl')

In [224]:
!ls

combine_and_clean_data.ipynb  reports
data			      simple_random_forest.ipynb
explore_raw_data.ipynb	      simple_random_forest_w_req_amount.ipynb
ideas.txt		      tsa-wait-times-january-2006-december-2015.xls
models			      web_app
pdf_to_csvs.ipynb


In [530]:
# df.to_pickle(f'{cleandir}/clean_for_classifier_df.pkl')

In [526]:
# joblib.dump(df, f'{cleandir}/clean_for_classifier_df.joblib')

['data/clean/clean_for_classifier_df.joblib']

In [164]:
df = df.dropna()


df['binary_disposition'] = df['disposition']
df['binary_disposition'] = df['disposition'].where(df['disposition'] == 'Deny', other='Compensate')

# Change some text to make it more human readable
df.claim_site[df.claim_site == '-'] = 'Unknown'
df.claim_type[df.claim_type == '-'] = 'Unknown'

# Feature Engineering

### Count of items claimed

# TODO: Consider only using this for where claim_type is related to property.
df['num_items_or_incidents_claimed'] = df['item_category'].str.split(pat=';').apply(lambda x: len(x))
df['num_items_or_incidents_claimed'] = df['num_items_or_incidents_claimed'].where(df['claim_type'].str.contains('property', case=False) == True, other= 0)

### Time calculation
wait_period = df.date_received - df.incident_date
df['days_waited_to_file_claim'] = wait_period.dt.days

# Drop days where the 'date_received" was reported before 'incident_date'
df = df[df.days_waited_to_file_claim >= 0]

df.reset_index(inplace=True,drop=True)
df['bin_dispos_onehot'] = df['binary_disposition'].apply(lambda x: 1 if x == 'Compensate' else 0)

/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [165]:
# df['Week/Year_inc_date'] = df['incident_date'].apply(lambda x: "%d/%d" % (x.week, x.year))
# df['Week_inc_date'] = df['incident_date'].apply(lambda x: "%d" % (x.week))
# df['Month_inc_date'] = df['incident_date'].apply(lambda x: "%d" % (x.month))
# df['Year_inc_date'] = df['incident_date'].apply(lambda x: "%d" % (x.year))

# df['Week_received'] = df['date_received'].apply(lambda x: "%d" % (x.week))
df['Month_received'] = df['date_received'].apply(lambda x: "%d" % (x.month))
# df['Year_received'] = df['date_received'].apply(lambda x: "%d" % (x.year))

# Pickle Lists for Web App

In [182]:
itemlist = list(sorted(df.item_category.str.replace(' ','').str.split(pat=';').apply(pd.Series).stack().unique()))
itemlist = itemlist[2:]

airportlist = list(sorted(df.airport_code.unique()))

airlinelist = list(sorted(df.airline.unique()))
airlinelist.remove('(OY)')
airlinelist.remove('-')
airlinelist.remove('Inc')
airlinelist.remove('Not Provided')
airlinelist.remove('vivaAerobus')

claim_types = list(sorted(df.claim_type.unique()))

claim_sites = list(sorted(df.claim_site.unique()))

In [201]:
featuredir = './web_app/featurelists'

In [203]:
joblib.dump(itemlist, f'{featuredir}/item_category.joblib')
joblib.dump(airportlist, f'{featuredir}/airports.joblib')
joblib.dump(airlinelist, f'{featuredir}/airlines.joblib')
joblib.dump(claim_types, f'{featuredir}/claim_types.joblib')
joblib.dump(claim_sites, f'{featuredir}/claim_sites.joblib')

['./web_app/featurelists/claim_sites.joblib']

In [215]:
!ls web_app/stat_models/

rf_full_month_rec.joblib
